In [3]:
%pip install --upgrade pip
%pip install --disable-pip-version-check \
    torch==1.13.1 \
    torchdata==0.5.1 --quiet
%pip install \
    transformers==4.27.2 \                  
    datasets==2.11.0 --quiet

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.16.0 requires torch==2.1.0, but you have torch 1.13.1 which is incompatible.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [7]:
from datasets import load_dataset
from transformers import AutoModelForSeq2SeqLM
from transformers import AutoTokenizer
from transformers import GenerationConfig

In [8]:
huggingface_dataset_name="knkarthick/dialogsum"
dataset=load_dataset(huggingface_dataset_name)

Found cached dataset csv (/home/outbreakkp/.cache/huggingface/datasets/knkarthick___csv/knkarthick--dialogsum-cd36827d3490488d/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1)
100%|██████████| 3/3 [00:00<00:00, 837.35it/s]


In [9]:
model_name='google/flan-t5-base'
model=AutoModelForSeq2SeqLM.from_pretrained(model_name)

In [10]:
tokenizer=AutoTokenizer.from_pretrained(model_name,use_fast=True)

No Prompt Engineering done


In [11]:
example_indices=[2]
for i,index in enumerate(example_indices):
    dialogue=dataset['test'][index]['dialogue']
    summary=dataset['test'][index]['summary']

    input=tokenizer(dialogue, return_tensors='pt')
    output=tokenizer.decode(
        model.generate(
            input["input_ids"],
            max_new_tokens=50,
        )[0],
        skip_special_tokens=True
    )
    print(dialogue)
    print(summary)
    print(output)

#Person1#: Ms. Dawson, I need you to take a dictation for me.
#Person2#: Yes, sir...
#Person1#: This should go out as an intra-office memorandum to all employees by this afternoon. Are you ready?
#Person2#: Yes, sir. Go ahead.
#Person1#: Attention all staff... Effective immediately, all office communications are restricted to email correspondence and official memos. The use of Instant Message programs by employees during working hours is strictly prohibited.
#Person2#: Sir, does this apply to intra-office communications only? Or will it also restrict external communications?
#Person1#: It should apply to all communications, not only in this office between employees, but also any outside communications.
#Person2#: But sir, many employees use Instant Messaging to communicate with their clients.
#Person1#: They will just have to change their communication methods. I don't want any - one using Instant Messaging in this office. It wastes too much time! Now, please continue with the memo. Wh

Zero Shot Inference

In [12]:
for i,index in enumerate(example_indices):
    dialogue=dataset['test'][index]['dialogue']
    summary=dataset['test'][index]['summary']

    prompt=f"""
Summarixer the following conversation
{dialogue}
Summary:
  """

    input=tokenizer(prompt, return_tensors='pt')
    output=tokenizer.decode(
        model.generate(
            input["input_ids"],
            max_new_tokens=50,
        )[0],
        skip_special_tokens=True
    )
    print(dialogue)
    print(summary)
    print(output)

#Person1#: Ms. Dawson, I need you to take a dictation for me.
#Person2#: Yes, sir...
#Person1#: This should go out as an intra-office memorandum to all employees by this afternoon. Are you ready?
#Person2#: Yes, sir. Go ahead.
#Person1#: Attention all staff... Effective immediately, all office communications are restricted to email correspondence and official memos. The use of Instant Message programs by employees during working hours is strictly prohibited.
#Person2#: Sir, does this apply to intra-office communications only? Or will it also restrict external communications?
#Person1#: It should apply to all communications, not only in this office between employees, but also any outside communications.
#Person2#: But sir, many employees use Instant Messaging to communicate with their clients.
#Person1#: They will just have to change their communication methods. I don't want any - one using Instant Messaging in this office. It wastes too much time! Now, please continue with the memo. Wh

Few Shot Inference

In [13]:
def make_prompt(dummy_prompt,indices_to_make_prompt):
    for index in indices_to_make_prompt:
        dialogue=dataset['test'][index]['dialogue']
        summary=dataset['test'][index]['summary']
        dummy_prompt+=f"""
Summarize the following conversation
{dialogue}
Summary:
{summary}
  """
        dialogue=dataset['test'][2]['dialogue']
        dummy_prompt+=f"""
Dialogue:
{dialogue}
Summary:
"""
        return  dummy_prompt

In [14]:
indices_to_make_prompt=[3,4]
prompt=""
prompt_new=make_prompt(prompt,indices_to_make_prompt)
input=tokenizer(prompt_new, return_tensors='pt')
output=tokenizer.decode(
        model.generate(
            input["input_ids"],
            max_new_tokens=50,
        )[0],
        skip_special_tokens=True
    )
print(dialogue)
print(summary)
print(output)

Token indices sequence length is longer than the specified maximum sequence length for this model (741 > 512). Running this sequence through the model will result in indexing errors


#Person1#: Ms. Dawson, I need you to take a dictation for me.
#Person2#: Yes, sir...
#Person1#: This should go out as an intra-office memorandum to all employees by this afternoon. Are you ready?
#Person2#: Yes, sir. Go ahead.
#Person1#: Attention all staff... Effective immediately, all office communications are restricted to email correspondence and official memos. The use of Instant Message programs by employees during working hours is strictly prohibited.
#Person2#: Sir, does this apply to intra-office communications only? Or will it also restrict external communications?
#Person1#: It should apply to all communications, not only in this office between employees, but also any outside communications.
#Person2#: But sir, many employees use Instant Messaging to communicate with their clients.
#Person1#: They will just have to change their communication methods. I don't want any - one using Instant Messaging in this office. It wastes too much time! Now, please continue with the memo. Wh